# Нейронные сети и компьютерное зрение

# 1. Задачи решаемые при помощи НН


## Бинарная классификация

+ Функция активации - сигмоида
+ Функция потерь - бинарная кросс-энтропия

## Многоклассовая классификация

+ Функция активации - софтмакс
+ Функция потерь - кросс-энтропия

## Локализация

![local](img/local.png)

![local](img/local_act.png)

## Сегментация

Задача - отделить пиксели где объект есть от пикселей где его нет. Сеть выдает скоры для каждого пикселя

+ Функция активации - сигмоида
+ Функция потерь - бинарная кросс-энтропия

![segment](img/segment.png)

## Сжатие размерности

+ Относится к обучению без учителя.
+ В Embedding слое должно быть значительно меньше параметров чем пикселей на изображение
+ По сжатому состоянию картинке (embedding) дальше можно решать задачи классификации

![embedding](img/embed.png)

## Super Resolution

+ увеличение или уменьшение разрешения картинки

![superresolution](img/superresolution.png)

# 2. Методы оптимизации

Невозможно заранее определить, какой из оптимизаторов покажет лучшие результаты 

+ **Cтохастический градиентный спуск (SGD)**
  + стохастический - вычисление градиента на части данных (батчи)
+ **Градиентный спуск с импульсом**
+ **Rprop**
  + изменяет скорость обучения в зависимости от градиента
  + плохо работает с батчами
+ **RMSprop**
  + Если вдоль направления производная болшая, то нужно идти помедленее и наоборот
+ **Adam**
  + хорошие результаты для большого класса НН
  + параметры как правило не нужно подбирать. Оптимальные α=3*10^-4, β1 = 0.9, β2 = 0.999

# 3. Сверточные нейронные сети

Преимущества по сравнению с полносвязными НН:
+ Инвариантны к положению объекта на картинке
+ Более легковесны
+ Учитывает структуру пикселей (соседство пикселей)

## Сверточный слой

Параметры свертки:
+ Шаг свертки (stride)
+ Отступ (Padding)
+ Размер ядра свертки (kernel size)
+ Количество входных каналов
+ Количество выходных каналов

1 канал

![conv](img/conv.png)

3 канала
![conv](img/conv_channel.png)

## Polling слой

Позволяет экономить память выкидывая часть информации

Разновидности:
+ MaxPool
+ MinPool
+ AvgPool

![polling](img/polling.png)

# 4. Архитекруры сверточных нейронных сетей

## LeNet (1998)

⚡ Придумана для распознавания одноканальных (чернобелых) рукописных цифр

⚡ Функция активации tanh, свёрточные слои и пулинг

![lenet](img/lenet.png)

## AlexNet (2012)

⚡ Каскад свёрток (увеличение receptive field) и функция активации ReLU

![AlexNet](img/alexnet.png)

## VGG (2014)

⚡ Обучать урезанную версию сети, постепенно добавляя слои, а также свёртки 1х1. Помогает бороться с затухающим градиентом

![AlexNet](img/vgg.png)

## GoogLeNet (2015)

⚡ В Inception blok применяются параллельные вычисления к картинке с последущей конкатенацией результатов

⚡ Слои, где собирается тензор из нескольких свёрток разного размера

⚡ Применяются вспомогательные функции потерь при затухании градиентов

![inception_block](img/inception_block.png)

![GoogLeNet](img/googlelenet.png)

## ResNet (2015)

⚡ Обходные соединения, пробрасывающие градиент ошибки в обход свёртки

⚡ Может принимать картинки любого размера за счет усреднения на выходе

![residual_block](img/residual_block.png)

![residual_block](img/resnet.png)

# 5. Регуляризация и нормализация

## Борьба с пререобучением:

1. упростить модель
2. увеличить количество данных

+ **Аугментация** - способ получить больше данных. Для картинок:
  + вырезать часть изображения
  + сжать/расширить изображение
  + отзеркалить
  + повернуть
  + поменять контраст, цвет, насыщенность
  + добавить шумы

3. Early stopping
4. Регуляризация
  + Тихонова (Ridge)
  + Lasso
5. **DropOut**
  + отключение некоторых связей в НН
  + зануление некоторых нейронов
  
## Нормализация

Нормализованные данные приводят к более быстрому обучению модели

### BatchNorm
https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html?highlight=eval

+ Можно использовать на каждом слое НН
+ Во время тренировки батч-нормализация оценивает статистики по одному батчу, а во время валидации используются усредненные статистики по всей истории обучения



# 6. Семинар: Реализация сверточного слоя 

In [1]:
import torch
import torch.nn as nn
import numpy as np
from abc import ABC, abstractmethod

## Через циклы

In [2]:
def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    def __call__(self, input_tensor):
      #инициализация выходного тензора
        output_tensor = torch.zeros(calc_out_shape(input_tensor.shape, self.out_channels, self.kernel_size, self.stride, 0))
        # цикл по картинкам
        for image in torch.arange(output_tensor.shape[0]):
            # цикл по входным каналам картинок
            for channel in torch.arange(input_tensor.shape[1]):
                # цикл по высоте выходного слоя
                for height in torch.arange(output_tensor.shape[2]):
                    # цикл по ширине выходного слоя
                    for width in torch.arange(output_tensor.shape[3]):
                        # делаем срез входного тензора
                        pre_tensor = input_tensor[image, channel, height*self.stride: height*self.stride+self.kernel_size, 
                                          width*self.stride:width*self.stride+self.kernel_size]
                        # цикл по выходным каналам
                        for out_kernel in torch.arange(output_tensor.shape[1]):
                            output_tensor[image, out_kernel, height, width] += (pre_tensor * self.kernel[out_kernel, channel]).sum()
        return output_tensor

# Корректность реализации определится в сравнении со стандартным слоем из pytorch.
# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dLoop))

True


## Через матричное перемножение

In [3]:
class Conv2dMatrix(ABCConv2d):
    # Функция преобразование кернела в матрицу нужного вида.
    def one_filter(self, torch_input, filter, output_height, output_width):
        image = torch_input[0, 0]
        rows = []
        # цикл по высоте выходного слоя
        for height in torch.arange(output_height):
            # цикл по ширине выходного слоя
            for width in torch.arange(output_width):
                # делаем маску нулей изображения 
                mask = torch.zeros_like(image)
                # заполняем маску знаениями ядра
                mask[height*self.stride: height*self.stride+self.kernel_size, 
                                  width*self.stride:width*self.stride+self.kernel_size] += filter
                # вытягиваем тензор в одну строку
                rows.append(mask.flatten().unsqueeze(0))
        # состыковываем строки весов ядра        
        w = torch.cat(rows, 0)
        return w


    def _unsqueeze_kernel(self, torch_input, output_height, output_width):
        filters = []
        # цикл по фильтрам ядра
        for filter in self.kernel[0]:
            w = self.one_filter(torch_input, filter, output_height, output_width)
            filters.append(w)
        # присоедиям горизонтально фильтры    
        kernel_unsqueezed = torch.cat(filters, 1)
        return kernel_unsqueezed

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        kernel_unsqueezed = self._unsqueeze_kernel(torch_input, output_height, output_width)
        # print(kernel_unsqueezed)
        result = kernel_unsqueezed @ torch_input.view((batch_size, -1)).permute(1, 0)
        return result.permute(1, 0).view((batch_size, self.out_channels,
                                          output_height, output_width))

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dMatrix))

True


## Вытягивание матрицы весов

In [4]:
class Conv2dMatrixV2(ABCConv2d):
    # Функция преобразования кернела в нужный формат.
    def _convert_kernel(self):
        converted_kernel = self.kernel.reshape(self.kernel.shape[0], -1)
        return converted_kernel

    # Функция преобразования входа в нужный формат.
    def _convert_input(self, torch_input, output_height, output_width):
        res = []
        # цикл по картинкам
        for image in torch.arange(torch_input.shape[0]):
            channel_flat = []
            # цикл по входным каналам картинок
            for channel in torch.arange(torch_input.shape[1]):
                # цикл по высоте выходного слоя
                for height in torch.arange(output_height):
                    # цикл по ширине выходного слоя
                    for width in torch.arange(output_width):
                        # делаем срез входного тензора
                        pre_tensor = torch_input[image, channel, height*self.stride: height*self.stride+self.kernel_size,
                                         width*self.stride:width*self.stride+self.kernel_size]
                
                        channel_flat.append(pre_tensor.flatten().unsqueeze(0))
            converted_image = torch.cat(channel_flat).flatten().unsqueeze(1)
            res.append(converted_image)
        converted_input = torch.cat(res, dim=1)
        return converted_input

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        converted_kernel = self._convert_kernel()
        converted_input = self._convert_input(torch_input, output_height, output_width)

        conv2d_out_alternative_matrix_v2 = converted_kernel @ converted_input
        return conv2d_out_alternative_matrix_v2.transpose(1,0).view(torch_input.shape[0],
                                                     self.out_channels, 
                                                     output_height,
                                                     output_width)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dMatrixV2))

True


# 7. Семинар. Слой нормализации

In [8]:
def custom_batch_norm1d(input_tensor, eps):
    normed_tensor = (input_tensor - torch.mean(input_tensor, dim=0)) / \
    (torch.sqrt(torch.var(input_tensor, dim=0, unbiased=False) + eps))
    return normed_tensor


input_tensor = torch.Tensor([[0.0, 0, 1, 0, 2], [0, 1, 1, 0, 10]])
batch_norm = nn.BatchNorm1d(input_tensor.shape[1], affine=False)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):

all_correct = True
for eps_power in range(10):
    eps = np.power(10., -eps_power)
    batch_norm.eps = eps
    batch_norm_out = batch_norm(input_tensor)
    custom_batch_norm_out = custom_batch_norm1d(input_tensor, eps)

    all_correct &= torch.allclose(batch_norm_out, custom_batch_norm_out)
    all_correct &= batch_norm_out.shape == custom_batch_norm_out.shape
print(all_correct)

True


In [10]:
input_size = 7
batch_size = 5
input_tensor = torch.randn(batch_size, input_size, dtype=torch.float)

eps = 1e-3

def custom_batch_norm1d(input_tensor, weight, bias, eps):
    mean = torch.mean(input_tensor,dim=0)
    var = torch.var(input_tensor, dim=0, unbiased=False)
    std = torch.sqrt(var + eps)            
    z = (input_tensor - mean) / (std)
    normed_tensor = z * weight + bias
    return normed_tensor

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm_out = batch_norm(input_tensor)
custom_batch_norm_out = custom_batch_norm1d(input_tensor, batch_norm.weight.data, batch_norm.bias.data, eps)
print(torch.allclose(batch_norm_out, custom_batch_norm_out) \
      and batch_norm_out.shape == custom_batch_norm_out.shape)

True


In [12]:
input_size = 3
batch_size = 5
eps = 1e-1


class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum
        self.running_mean = torch.zeros(input_size)
        self.running_var = torch.ones(input_size)
        self.training = True

    def __call__(self, input_tensor):
        if self.training is False:
            mean = self.running_mean
            var = self.running_var
        else:
            mean = torch.mean(input_tensor,dim=0)
            var = torch.var(input_tensor, dim=0, unbiased=False)
            self.running_mean = self.momentum * mean + (1-self.momentum) * self.running_mean
            self.running_var = self.momentum * var * (batch_size/(batch_size-1)) + (1 - self.momentum) * \
            self.running_var 

        std = torch.sqrt(var  + self.eps)    
        z = (input_tensor - mean) / (std)
        normed_tensor = z * self.weight + self.bias  
        return normed_tensor
            

    def eval(self):
        # В этом методе реализуйте переключение в режим предикта.
        self.training = False


batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-06) \
        and norm_output.shape == custom_output.shape

batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-06) \
        and norm_output.shape == custom_output.shape
print(all_correct)

True


In [13]:
eps = 1e-3

input_channels = 3
batch_size = 3
height = 10
width = 10

batch_norm_2d = nn.BatchNorm2d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, height, width, dtype=torch.float)


def custom_batch_norm2d(input_tensor, eps):
    output_tensor = torch.zeros_like(input_tensor)
    for c in range(input_tensor.shape[1]):
        mean_ = torch.mean(input_tensor[:,c,:,:])
        vars_ = torch.var(input_tensor[:,c,:,:], unbiased=False)
        std_ = torch.sqrt(vars_ + eps)
        output_tensor[:,c,:,:] = (input_tensor[:,c,:,:] - mean_) / std_

    normed_tensor = output_tensor * 1 + 0
#     #Лучшее решение
#     mean = input_tensor.mean(dim=(0,2,3), keepdim=True)
#     var = (input_tensor-mean).pow(2).mean(dim=(0,2,3), keepdim=True)
#     normed_tensor = (input_tensor-mean)/torch.sqrt(var+eps)

    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = batch_norm_2d(input_tensor)
# print('НОРМ', norm_output)
custom_output = custom_batch_norm2d(input_tensor, eps)
# print('МОЙ', custom_output)
print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)

True


In [14]:
eps = 1e-10


def custom_layer_norm(input_tensor, eps):
    mean = input_tensor.mean(dim=tuple(range(1, len(input_tensor.shape))), keepdim=True)
    var = (input_tensor-mean).pow(2).mean(dim=tuple(range(1, len(input_tensor.shape))), keepdim=True)
    normed_tensor = (input_tensor-mean)/torch.sqrt(var+eps)
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for dim_count in range(3, 9):
    input_tensor = torch.randn(*list(range(3, dim_count + 2)), dtype=torch.float)
    layer_norm = nn.LayerNorm(input_tensor.size()[1:], elementwise_affine=False, eps=eps)

    norm_output = layer_norm(input_tensor)
    custom_output = custom_layer_norm(input_tensor, eps)

    all_correct &= torch.allclose(norm_output, custom_output, 1e-2)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True


In [15]:
eps = 1e-3

batch_size = 5
input_channels = 2
input_length = 30

instance_norm = nn.InstanceNorm1d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, input_length, dtype=torch.float)


def custom_instance_norm1d(input_tensor, eps):
    mean = input_tensor.mean(dim=(2), keepdim=True)
    var = (input_tensor-mean).pow(2).mean(dim=(2), keepdim=True)
    normed_tensor = (input_tensor-mean)/torch.sqrt(var+eps)
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
norm_output = instance_norm(input_tensor)
custom_output = custom_instance_norm1d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output, atol=1e-06) and norm_output.shape == custom_output.shape)

True


https://arxiv.org/pdf/1803.08494.pdf

In [16]:
channel_count = 6
eps = 1e-3
batch_size = 20
input_size = 2

input_tensor = torch.randn(batch_size, channel_count, input_size)


def custom_group_norm(input_tensor, groups, eps):
    N, C, L = input_tensor.shape
    x = torch.reshape(input_tensor, [N, groups, C // groups, L])
    mean = x.mean(dim=(2, 3), keepdim=True)
    var = (x-mean).pow(2).mean(dim=(2, 3), keepdim=True)
    normed_tensor = ((x-mean)/torch.sqrt(var+eps)).reshape(N, C, L)
    return normed_tensor


# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
all_correct = True
for groups in [1, 2, 3, 6]:
    group_norm = nn.GroupNorm(groups, channel_count, eps=eps, affine=False)
    norm_output = group_norm(input_tensor)
    custom_output = custom_group_norm(input_tensor, groups, eps)
    all_correct &= torch.allclose(norm_output, custom_output, 1e-3)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True


# 8. Наименьшее затухание градиента

In [3]:
seed = int(input())
np.random.seed(seed)
torch.manual_seed(seed)

NUMBER_OF_EXPERIMENTS = 200

class SimpleNet(torch.nn.Module):
    def __init__(self, activation):
        super().__init__()

        self.activation = activation
        self.fc1 = torch.nn.Linear(1, 1, bias=False)  # one neuron without bias
        self.fc1.weight.data.fill_(1.)  # init weight with 1
        self.fc2 = torch.nn.Linear(1, 1, bias=False)
        self.fc2.weight.data.fill_(1.)
        self.fc3 = torch.nn.Linear(1, 1, bias=False)
        self.fc3.weight.data.fill_(1.)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        return x

    def get_fc1_grad_abs_value(self):
        return torch.abs(self.fc1.weight.grad)

def get_fc1_grad_abs_value(net, x):
    output = net.forward(x)
    output.backward()  # no loss function. Pretending that we want to minimize output
                       # In our case output is scalar, so we can calculate backward
    fc1_grad = net.get_fc1_grad_abs_value().item()
    net.zero_grad()
    return fc1_grad


activation =  {'ELU': torch.nn.ELU(), 'Hardtanh': torch.nn.Hardtanh(),
               'LeakyReLU': torch.nn.LeakyReLU(), 'LogSigmoid': torch.nn.LogSigmoid(),
               'PReLU': torch.nn.PReLU(), 'ReLU': torch.nn.ReLU(), 'ReLU6': torch.nn.ReLU6(),
               'RReLU': torch.nn.RReLU(), 'SELU': torch.nn.SELU(), 'CELU': torch.nn.CELU(),
               'Sigmoid': torch.nn.Sigmoid(), 'Softplus': torch.nn.Softplus(),
               'Softshrink': torch.nn.Softshrink(), 'Softsign': torch.nn.Softsign(),
               'Tanh': torch.nn.Tanh(), 'Tanhshrink': torch.nn.Tanhshrink(),
               'Hardshrink': torch.nn.Hardshrink()}

for name, activation in activation.items():
    print(name)
    net = SimpleNet(activation=activation)

    fc1_grads = []
    for x in torch.randn((NUMBER_OF_EXPERIMENTS, 1)):
        fc1_grads.append(get_fc1_grad_abs_value(net, x))

    # Проверка осуществляется автоматически, вызовом функции:
    print(np.mean(fc1_grads))
    # (раскомментируйте, если решаете задачу локально)

42
ELU
0.45622111071075777
Hardtanh
0.34066947680839804
LeakyReLU
0.4554302493817175
LogSigmoid
0.25474448641529307
PReLU
0.420650872474871
ReLU
0.38253963836003096
ReLU6
0.3814026044867933
RReLU
0.3783517877686245
SELU
0.6172542834468185
CELU
0.519119741236791
Sigmoid
0.007594654063323106
Softplus
0.25964870156425607
Softshrink
0.20243875443935394
Softsign
0.06452517481287941
Tanh
0.16103094387799502
Tanhshrink
0.02247800910559467
Hardshrink
0.6688168460130691
